# 33. 装饰器（Decorators）

装饰器用于在不改动原函数代码的情况下扩展行为（日志、计时、缓存、重试、鉴权等）。本节讲清本质、wraps、带参数装饰器与常用模式。

> 约定：Python 3.8；示例尽量只用标准库；代码块可直接运行。


## 前置知识

- 第 16 节：函数高级话题（wraps/闭包）


## 知识点地图

- 1. 最小装饰器：decorator(fn) -> wrapper
- 2. 带参数装饰器：repeat(n)
- 3. 缓存：functools.lru_cache
- 4. 重试装饰器（示例）：retry(n)
- 5. 类装饰器（了解）：修改类或返回新类


## 自检清单（学完打勾）

- [ ] 理解装饰器本质：接收函数并返回新函数
- [ ] 会用 functools.wraps 保留元信息
- [ ] 会写带参数的装饰器（装饰器工厂）
- [ ] 会用 lru_cache 做缓存（纯函数场景）
- [ ] 了解类装饰器/方法装饰器（入门）


## 知识点 1：最小装饰器：decorator(fn) -> wrapper

装饰器本质就是一个高阶函数：接收函数并返回新函数。


In [ ]:
from functools import wraps

def debug(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        print('call', fn.__name__, args, kwargs)
        return fn(*args, **kwargs)
    return wrapper

@debug
def add(a, b):
    return a + b

print(add(1, 2))
print(add.__name__)


## 知识点 2：带参数装饰器：repeat(n)

带参数装饰器需要两层函数：repeat(n) -> deco(fn) -> wrapper。


In [ ]:
from functools import wraps

def repeat(n):
    def deco(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            result = None
            for _ in range(n):
                result = fn(*args, **kwargs)
            return result
        return wrapper
    return deco

@repeat(3)
def hello(name):
    print('hi', name)
    return name

print('return:', hello('Ada'))


## 知识点 3：缓存：functools.lru_cache

lru_cache 适合纯函数，入参必须可哈希。


In [ ]:
from functools import lru_cache

@lru_cache(maxsize=None)
def fib(n):
    if n < 2:
        return n
    return fib(n-1) + fib(n-2)

print(fib(30))


## 知识点 4：重试装饰器（示例）：retry(n)

典型模式：捕获指定异常，重试若干次；最后仍失败则抛出。


In [ ]:
from functools import wraps

class FlakyError(Exception):
    pass

def retry(times, exc=Exception):
    def deco(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            last = None
            for _ in range(times):
                try:
                    return fn(*args, **kwargs)
                except exc as e:
                    last = e
            raise last
        return wrapper
    return deco

counter = {'n': 0}

@retry(3, exc=FlakyError)
def flaky():
    counter['n'] += 1
    if counter['n'] < 3:
        raise FlakyError('fail')
    return 'ok'

print(flaky())


## 知识点 5：类装饰器（了解）：修改类或返回新类

类装饰器可以给类加属性/方法，也可以包装 __init__。


In [ ]:
import time

def with_created_at(cls):
    cls.created_at = time.time()
    return cls

@with_created_at
class A:
    pass

print(hasattr(A, 'created_at'))


## 常见坑

- 装饰器会改变调用栈与调试体验（必要时保留原函数信息）
- 装饰器应尽量保持“透明”：参数/返回值语义不变


## 综合小案例：实现 require_positive：参数校验装饰器

装饰器校验第一个参数必须为正数，否则抛 ValueError（演示简单校验模式）。


In [ ]:
from functools import wraps

def require_positive(fn):
    @wraps(fn)
    def wrapper(x, *args, **kwargs):
        if x <= 0:
            raise ValueError('x must be > 0')
        return fn(x, *args, **kwargs)
    return wrapper

@require_positive
def square(x):
    return x * x

print(square(3))
try:
    square(-1)
except Exception as e:
    print(type(e).__name__, e)


## 自测题（不写代码也能回答）

- 为什么写装饰器要用 wraps？
- 带参数装饰器为什么需要两层函数？
- lru_cache 对入参有什么要求？


## 练习题（建议写代码）

- 实现 timer 装饰器：打印耗时并返回结果。
- 实现 once 装饰器：函数只执行一次，之后返回第一次结果。
